# Set path and dependencies

In [ ]:
import os

In [2]:
# project specific paths
Tensorflow_path = 'Tensorflow'
project_work_space = 'Tensorflow/power_electronic_workspace'

image_path = project_work_space + '/image'
annotation_path = project_work_space + '/annotations'
project_models_path = project_work_space + '/my_models'
pre_trained_model_path = project_work_space + '/pre_trained_models'
scripts_path = Tensorflow_path + '/scripts'
apimodel_path = Tensorflow_path + '/models'

my_model_path = project_work_space + '/my_models'
config_path = my_model_path +'/power_electronic_mobnet/pipeline.config'
checkpoint_path = my_model_path+'/power_electronic_mobnet/'


# Dataset organization
split data into training set and validation set

In [4]:
train_val_split = 0.9
label_lst = ['connect','faulty']

In [6]:
for label in label_lst:
    count = 0
    split = len(os.listdir(os.path.join(image_path,label)))*train_val_split

    for file in os.listdir(os.path.join(image_path,label)):
        # print(file)
        if count < split:
            print('train',count,file)
            os.replace(os.path.join(image_path,label,file),os.path.join(image_path,'train',file))
            count += 1
        elif count > split: 
            # print(label)
            print('test',count,file)
            os.replace(os.path.join(image_path,label,file),os.path.join(image_path,'test',file))
            count+=1
        # break

train 0 connect0.jpg
train 1 connect0.xml
train 2 connect1.jpg
train 3 connect1.xml
train 4 connect10.jpg
train 5 connect10.xml
train 6 connect11.jpg
train 7 connect11.xml
train 8 connect12.jpg
train 9 connect12.xml
train 10 connect13.jpg
train 11 connect13.xml
train 12 connect14.jpg
train 13 connect14.xml
train 14 connect15.jpg
train 15 connect15.xml
train 16 connect16.jpg
train 17 connect16.xml
train 18 connect17.jpg
train 19 connect17.xml
train 20 connect18.jpg
train 21 connect18.xml
train 22 connect19.jpg
train 23 connect19.xml
train 24 connect2.jpg
train 25 connect2.xml
train 26 connect20.jpg
train 27 connect20.xml
train 28 connect21.jpg
train 29 connect21.xml
train 30 connect22.jpg
train 31 connect22.xml
train 32 connect23.jpg
train 33 connect23.xml
train 34 connect24.jpg
train 35 connect24.xml
train 36 connect25.jpg
train 37 connect25.xml
train 38 connect26.jpg
train 39 connect26.xml
train 40 connect27.jpg
train 41 connect27.xml
train 42 connect28.jpg
train 43 connect28.xml
trai

# Creating label map

In [7]:
labels_map = [{'name':'connect','id':1},{'name':'faulty','id':2}]

In [8]:
with open(annotation_path + '/label_map.pbtxt','w') as file:
    for label in labels_map:
        file.write('item { \n')
        file.write('\tname:\'{}\'\n'.format(label['name']))
        file.write('\tid:{}\n'.format(label['id']))
        file.write('}\n')


# TF RECORDS
Converting *.xml to *.records
script and code provided: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records

In [11]:
# # Create train data:
!python {scripts_path + '/preprocessing/generate_tfrecord.py'} -x {image_path + '/train'} -l {annotation_path + '/label_map.pbtxt'} -o {annotation_path + '/train.record'}


# # Create test data:
!python {scripts_path + '/preprocessing/generate_tfrecord.py'} -x {image_path + '/test'} -l {annotation_path + '/label_map.pbtxt'} -o {annotation_path + '/test.record'}

Successfully created the TFRecord file: Tensorflow/power_electronic_workspace/annotations/test.record


# Download the pretrained model zip from TF zoo

In [ ]:
# model_link = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
# !wget.download(model_link)

# Transfer learning

In [14]:
CUSTOM_MODEL_NAME = 'power_electronic_mobnet' 

In [3]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [19]:
config_path = my_model_path+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
config = config_util.get_configs_from_pipeline_file(config_path)
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [20]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(config_path, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [22]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 8
pipeline_config.train_config.fine_tune_checkpoint = pre_trained_model_path+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= annotation_path + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [annotation_path + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = annotation_path + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [annotation_path + '/test.record']

In [23]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

In [38]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(apimodel_path, project_models_path,CUSTOM_MODEL_NAME,project_models_path,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/power_electronic_workspace/my_models/power_electronic_mobnet --pipeline_config_path=Tensorflow/power_electronic_workspace/my_models/power_electronic_mobnet/pipeline.config --num_train_steps=5000


# Build the model using the chechpoints created

In [4]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [5]:
config = config_util.get_configs_from_pipeline_file(config_path)
detection_model = model_builder.build(model_config = config['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(checkpoint_path, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Use OpenCV to detect faulty power electronics in real time

In [6]:
import cv2 
import numpy as np

In [7]:
category_index = label_map_util.create_category_index_from_labelmap(annotation_path+'/label_map.pbtxt')

In [9]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [10]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

ValueError: in user code:

    File "C:\Users\User\AppData\Local\Temp\ipykernel_19036\1707628044.py", line 10, in detect_fn  *
        image, shapes = detection_model.preprocess(image)
    File "c:\Users\User\anaconda3\envs\tf2\lib\site-packages\object_detection\meta_architectures\ssd_meta_arch.py", line 485, in preprocess  *
        normalized_inputs, self._image_resizer_fn)
    File "c:\Users\User\anaconda3\envs\tf2\lib\site-packages\object_detection\utils\shape_utils.py", line 492, in resize_images_and_return_shapes  *
        outputs = static_or_dynamic_map_fn(
    File "c:\Users\User\anaconda3\envs\tf2\lib\site-packages\object_detection\utils\shape_utils.py", line 246, in static_or_dynamic_map_fn  *
        outputs = [fn(arg) for arg in tf.unstack(elems)]
    File "c:\Users\User\anaconda3\envs\tf2\lib\site-packages\object_detection\core\preprocessor.py", line 3330, in resize_image  *
        new_image = tf.image.resize_images(

    ValueError: 'images' must have either 3 or 4 dime

: 